In [15]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import numpy as np

import os

from dotenv import load_dotenv

load_dotenv()

# Brave Search API configuration
BRAVE_SEARCH_API_KEY = os.environ.get("BRAVE_SEARCH_API_KEY")
BRAVE_SEARCH_ENDPOINT = "https://api.search.brave.com/res/v1/web/search"

# OpenAI API configuration
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


# 1. Web Search Integration using Brave Search
def brave_search(query, count=5):
    headers = {
        "Accept": "application/json",
        "X-Subscription-Token": BRAVE_SEARCH_API_KEY
    }
    params = {
        "q": query,
        "count": count
    }
    response = requests.get(BRAVE_SEARCH_ENDPOINT, headers=headers, params=params)
    return response.json()['web']['results']

# 2. Web Scraping and Content Extraction
def extract_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text(separator=' ', strip=True)

# 3. Text Processing
def preprocess_text(text, max_length=10000):
    # Simple preprocessing: truncate to max_length characters
    return text[:max_length]

# 4. Embedding Generation using OpenAI
def generate_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding

# 5. Retrieval
def retrieve_relevant_info(query_embedding, doc_embeddings, docs, top_k=3):
    similarities = np.dot(doc_embeddings, query_embedding)
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [docs[i] for i in top_indices]

# 6 & 7. Prompt Engineering and Language Model Integration
def generate_answer(query, relevant_info):
    prompt = f"""Given the following information, please provide a concise and informative answer to the query.

Query: {query}

Relevant Information:
{relevant_info}

Answer:"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides accurate and concise information based on the given query, chat history, and relevant information retrieved."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150
    )
    return response.choices[0].message.content.strip()

# Main RAG function
def rag_web_search(query):
    # Perform web search
    search_results = brave_search(query)
    
    # Extract and process content
    docs = []
    for result in search_results:
        content = extract_content(result['url'])
        processed_content = preprocess_text(content)
        docs.append(processed_content)
    
    # Generate embeddings
    doc_embeddings = [generate_embedding(doc) for doc in docs]
    query_embedding = generate_embedding(query)
    
    # Retrieve relevant information
    relevant_info = retrieve_relevant_info(query_embedding, doc_embeddings, docs)
    
    # Generate final answer
    answer = generate_answer(query, "\n".join(relevant_info))
    
    return answer
    
rag_web_search(input('Enter your query: '))

KeyboardInterrupt: Interrupted by user